In [ ]:
import pandas as pd
import ast  

In [ ]:
 df = pd.read_csv(r"C:\Users\EMRE\Desktop\nlp_proje\train.csv")

In [ ]:
def safe_parse(convo_str):
    try:
     
     return json.loads(convo_str.replace("'", '"'))
    except Exception as e:
        return None 

In [ ]:
df["parsed_conversations"] = df["conversations"].apply(safe_parse) 

In [ ]:
 df = df[df["parsed_conversations"].notnull()] 

In [ ]:
 prompts = []
responses = []

for convo in df["parsed_conversations"]:
  for i in range(len(convo) - 1):
    if convo[i]["from"] == "human" and convo[i + 1]["from"] == "gpt":
        prompts.append(convo[i]["value"])
        responses.append(convo[i + 1]["value"])

In [ ]:
 chat_df = pd.DataFrame({
"prompt": prompts,
"response": responses
})

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EMRE\Desktop\nlp_proje\train.csv")

In [ ]:
prompts = []
responses = []

def parse_conversation(raw_text):
    try:
# Sözlük listesi olarak ayrıştır
     convo = ast.literal_eval(raw_text)
     for i in range(len(convo)-1):
          if convo[i]["from"] == "human" and convo[i+1]["from"] == "gpt":
           prompts.append(convo[i]["value"])
           responses.append(convo[i+1]["value"])
    except Exception as e:
      pass # hatalı satır varsa atla  sonraki df["conversations"].apply(parse_conversation)

In [ ]:
data = pd.DataFrame({"prompt": prompts, "response": responses})
print(data.head())
print(f"Toplam {len(data)} adet soru-cevap çifti çıkarıldı.")

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\EMRE\Desktop\nlp_proje\train.csv")
print(df["conversations"].iloc[0])
print()
print(type(df["conversations"].iloc[0]))

In [ ]:
df = pd.read_csv(r"C:\Users\EMRE\Desktop\nlp_proje\train.csv")
def fix_json_string(s):
# '}{' aralarına virgül ekle → '}, {'
 s_fixed = re.sub(r'}\s*{', '}, {', s.strip())
 return ast.literal_eval(s_fixed)

In [ ]:
import pandas as pd
import re # ← BU SATIR EKLENDİ
import ast
df["conversations_fixed"] = df["conversations"].apply(fix_json_string)

In [ ]:
for i in range(2):
 print(f"--- Satır {i} ---")
for turn in df["conversations_fixed"].iloc[i]:
 print(f"{turn['from']}: {turn['value']}")
 print("\n")

In [ ]:
 from tqdm import tqdm

In [ ]:
 pairs = []

In [ ]:
for conv in tqdm(df["conversations_fixed"]):
 for i in range(len(conv) - 1):
  if conv[i]["from"] == "human" and conv[i + 1]["from"] == "gpt":
   prompt = conv[i]["value"].strip()
   response = conv[i + 1]["value"].strip()
   pairs.append((prompt, response))

In [ ]:
 final_df = pd.DataFrame(pairs, columns=["prompt", "response"])

In [ ]:
print(final_df.sample(5))

In [ ]:
final_df.to_csv("empathy_chat_data.csv", index=False) 

In [ ]:
 pip install faiss-cpu sentence-transformers  

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import pickle 

In [ ]:
import ast
import pandas as pd

In [ ]:
def fix_json_string(s):
 import re
 s_fixed = re.sub(r'}\s*{', '}, {', s.strip())
 return ast.literal_eval(s_fixed) 

In [ ]:
data = []
for i, row in df.iterrows():
    try:
     convo = fix_json_string(row["conversations"])
     for j in range(len(convo) - 1):
      if convo[j]["from"] == "human" and convo[j+1]["from"] == "gpt":
       prompt = convo[j]["value"].strip()
       response = convo[j+1]["value"].strip()
       data.append({"prompt": prompt, "response": response})
    except Exception as e:
     continue # bozuk satırları atla

df_ready = pd.DataFrame(data) 

In [ ]:
df_rag = df_ready.copy() 

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
corpus_embeddings = model.encode(df_rag["prompt"].tolist(), convert_to_numpy=True, show_progress_bar=True) 

In [ ]:
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings) 

In [ ]:
faiss.write_index(index, "rag_index.faiss")
with open("rag_data.pkl", "wb") as f:
    pickle.dump(df_rag, f) 

In [ ]:
import pickle
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
# FAISS index'i ve dataframe'i yükle
index = faiss.read_index("rag_index.faiss")
with open("rag_data.pkl", "rb") as f:
    df_rag = pickle.load(f)

# Aynı embedding modelini yeniden yükle
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
def rag_chatbot(user_input, top_k=3):
    query_embedding = model.encode([user_input], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    
    print("\n--- En İlgili Cevaplar ---")
    for i in range(top_k):
        print(f"\n[+] Soru: {df_rag.iloc[indices[0][i]]['prompt']}")
        print(f"[=] Cevap: {df_rag.iloc[indices[0][i]]['response']}")  

In [ ]:
rag_chatbot("Son zamanlarda kendimi yalnız ve umutsuz hissediyorum. Ne yapmalıyım?")  

In [ ]:
pip install sentence-transformers faiss-cpu pandas


In [ ]:
import pandas as pd
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Dosyaları yükle
feedback_df = pd.read_csv("feedback_data.csv")

with open("rag_data.pkl", "rb") as f:
    df_rag = pickle.load(f)

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Geri bildirimleri uygula
for _, row in feedback_df.iterrows():
    prompt = row["prompt"]
    original = row["original_response"]
    improved = row["improved_response"]

    mask = (df_rag["prompt"] == prompt) & (df_rag["response"] == original)
    df_rag.loc[mask, "response"] = improved

# Yeni embedding'leri üret
embeddings = model.encode(df_rag["prompt"].tolist(), show_progress_bar=True)

# FAISS index’i yeniden oluştur
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Güncellenmiş dosyaları kaydet
with open("rag_data.pkl", "wb") as f:
    pickle.dump(df_rag, f)

faiss.write_index(index, "rag_index.faiss")


In [ ]:
import gradio as gr
import pickle
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from deep_translator import GoogleTranslator
import re
import os

# Embed modeli
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Veri yolları
rag_path = "rag_data.pkl"
index_path = "rag_index.faiss"
feedback_path = "feedback_data.csv"

# Başlangıçta FAISS ve RAG verisi yükleniyor
with open(rag_path, "rb") as f:
    df_rag = pickle.load(f)
index = faiss.read_index(index_path)

# Çeviri fonksiyonu
def translate(text, src='auto', tgt='en'):
    try:
        return GoogleTranslator(source=src, target=tgt).translate(text)
    except Exception:
        return "[Çeviri Hatası]"

# İsim temizleyici
def clean_name(text):
    text = re.sub(r"\b(Hi|Hello|Hey)\s+Charlie\b[:,]?", "", text)
    text = re.sub(r"\bCharlie\b[:,]?", "", text)
    return text.strip()

# Nova yanıt üretici
def rag_chatbot(user_input):
    translated_input = translate(user_input, src="tr", tgt="en")
    query_embedding = model.encode([translated_input])
    D, I = index.search(np.array(query_embedding), k=3)
    response_en = df_rag.iloc[I[0][0]]["response"]
    response_en = clean_name(response_en)
    response_tr = translate(response_en, src="en", tgt="tr")
    return response_tr

# Geri bildirim kaydedici
def save_feedback(original_prompt, nova_response, improved_response):
    feedback_entry = {
        "prompt": original_prompt,
        "original_response": nova_response,
        "improved_response": improved_response
    }
    if os.path.exists(feedback_path):
        df_feedback = pd.read_csv(feedback_path)
        df_feedback = pd.concat([df_feedback, pd.DataFrame([feedback_entry])], ignore_index=True)
    else:
        df_feedback = pd.DataFrame([feedback_entry])
    df_feedback.to_csv(feedback_path, index=False)
    return "Geri bildiriminiz kaydedildi. Teşekkürler!"

# Geri bildirim sonrası görünür mesaj gösterici
def save_feedback_and_show_status(prompt, response, improved):
    message = save_feedback(prompt, response, improved)
    return gr.update(value=message, visible=True)

# Geri bildirimlere göre veriyi güncelle (sadece değişen satırlar embed edilir)
def update_rag_data():
    try:
        if not os.path.exists(feedback_path) or not os.path.exists(rag_path):
            return "Gerekli dosyalar bulunamadı."

        df_feedback = pd.read_csv(feedback_path)
        with open(rag_path, "rb") as f:
            df_rag_local = pickle.load(f)

        updated_prompts = []
        for _, row in df_feedback.iterrows():
            prompt = row["prompt"]
            original = row["original_response"]
            improved = row["improved_response"]

            mask = (df_rag_local["prompt"] == prompt) & (df_rag_local["response"] == original)
            if mask.any():
                idx = df_rag_local[mask].index[0]
                if df_rag_local.at[idx, "response"] != improved:
                    df_rag_local.at[idx, "response"] = improved
                    updated_prompts.append(prompt)
            else:
                new_row = {"prompt": prompt, "response": improved}
                df_rag_local = pd.concat([df_rag_local, pd.DataFrame([new_row])], ignore_index=True)
                updated_prompts.append(prompt)

        if updated_prompts:
            new_embeddings = model.encode(updated_prompts, show_progress_bar=False)
            index.add(np.array(new_embeddings))

            # Global veri güncelle
            global df_rag
            df_rag = df_rag_local

            # Güncellenmiş dosyaları kaydet
            with open(rag_path, "wb") as f:
                pickle.dump(df_rag_local, f)
            faiss.write_index(index, index_path)

            return f"{len(updated_prompts)} cevap güncellendi ✅"
        else:
            return "Yeni güncellenecek içerik bulunamadı."

    except Exception as e:
        return f"Hata oluştu: {str(e)}"

# Gradio Arayüz
with gr.Blocks(title="Nova - Empatik Chatbot") as demo:
    gr.Markdown(
        """
        # 🤖 Nova  
        Kısa, empatik, Türkçe yanıtlar sunan bir sohbet arkadaşı.  
        Kendini ifade et, Nova seni dinliyor.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            user_input = gr.Textbox(label="Mesajın", placeholder="Bugün nasıl hissediyorsun?", lines=4)
            submit_btn = gr.Button("Nova'ya Sor")
        with gr.Column(scale=5):
            chatbot_output = gr.Textbox(label="Nova'nın Cevabı", lines=5, interactive=False)

    with gr.Row():
        update_button = gr.Button("🔁 Cevapları Güncelle")
        update_status = gr.Textbox(label="", interactive=False, visible=True)

    # Geri bildirim alanı
    with gr.Accordion("Nova'nın cevabını geliştirmek ister misin?", open=False):
        improved_reply = gr.Textbox(label="Daha iyi bir önerin varsa buraya yaz.")
        submit_feedback_btn = gr.Button("Geri Bildirim Gönder")
        feedback_status = gr.Textbox(label="", interactive=False, visible=False)

    submit_btn.click(fn=rag_chatbot, inputs=user_input, outputs=chatbot_output)
    submit_feedback_btn.click(
        fn=save_feedback_and_show_status,
        inputs=[user_input, chatbot_output, improved_reply],
        outputs=feedback_status
    )
    update_button.click(fn=update_rag_data, inputs=[], outputs=update_status)

demo.launch()
